# Video Director — Script Generation Walkthrough

This notebook demonstrates the video script director, which uses an LLM to generate
segmented video scripts from project data.

1. **Browse Projects** — Pick a project from the database
2. **Generate Script** — Call the LLM director
3. **Inspect Output** — Review segments, narration, and FLUX scene descriptions
4. **Try Another** — Generate for a different project to see variety

## Setup

In [1]:
import sys
sys.path.insert(0, '/app')

import nest_asyncio
nest_asyncio.apply()

import asyncio
import json

## 1. Browse Projects

Load some projects from the database and pick one to generate a video script for.

In [2]:
from src.db.projects import get_all_projects

projects = get_all_projects(limit=10, is_valid=True)
print(f"Found {len(projects)} projects:\n")
for p in projects:
    print(f"  [{p.id}] {p.title}")
    print(f"       {p.short_description}")
    print(f"       Tags: {', '.join(p.hashtags)}")
    print()

Found 10 projects:

  [17] Gaming Portal with 200+ Games
       Collection of 200+ browser games built with libGDX and threejs
       Tags: gaming, libgdx, threejs, indiegames, webgames

  [16] Free USCIS Form Filler
       Browser-based USCIS form filler with PDF export
       Tags: uscis, pdf, form, web, free

  [15] Ruby Port to Fil-C
       Translating Ruby runtime to Fil-C language
       Tags: ruby, fil-c, porting, language

  [14] Actualize This: Maslow Card Game
       A card game that gamifies Maslow's hierarchy
       Tags: cardgame, maslow, gamification, philosophy, strategy

  [13] Museum Postcard Maker
       Web tool to create postcards from museum artworks
       Tags: museums, postcards, creativecommons, art, webapp

  [12] Firefox New Tab Art Add‑on
       Adds artistic new‑tab backgrounds to Firefox
       Tags: firefox, extension, art, newtab

  [11] New Tab New Art Chrome Extension
       Provides fresh artwork on each new tab
       Tags: chrome, extension, art, ne

## 2. Select a Project

Change `PROJECT_ID` below to the project you want to generate a script for.

In [3]:
from src.db.projects import get_project

PROJECT_ID = projects[0].id if projects else 1  # <-- change this

project = get_project(PROJECT_ID)
assert project is not None, f"Project {PROJECT_ID} not found"

print(f"Title:       {project.title}")
print(f"Summary:     {project.short_description}")
print(f"Description: {project.description}")
print(f"Tags:        {', '.join(project.hashtags)}")
print(f"Idea score:  {project.idea_score}/10")
print(f"Complexity:  {project.complexity_score}/10")
if project.url_summaries:
    print(f"URLs:")
    for url, summary in project.url_summaries.items():
        print(f"  {url}: {summary}")

Title:       Gaming Portal with 200+ Games
Summary:     Collection of 200+ browser games built with libGDX and threejs
Description: The project is a gaming portal featuring over 200 games developed by the creator over the years, all built using either libGDX or threejs frameworks. It serves as a showcase of the developer's portfolio and provides free-to-play browser-based games without monetization plans currently. The creator recently gained Google AdSense approval for experimental ad integration while focusing on traffic growth and SEO improvements. This platform highlights indie game development capabilities and offers diverse gaming experiences across multiple titles.
Tags:        gaming, libgdx, threejs, indiegames, webgames
Idea score:  7/10
Complexity:  6/10
URLs:
  https://example.com: The gaming portal hosts over 200 browser games developed with libGDX and threejs frameworks, featuring a portfolio of indie-developed titles without current monetization plans.


## 3. Generate Video Script

Call the director to generate a video script. Uses the creative LLM config (temperature=0.8)
for varied output.

In [4]:
from src.video_director import generate_video_script

async def generate():
    return await generate_video_script(
        title=project.title,
        short_description=project.short_description,
        description=project.description,
        hashtags=project.hashtags,
        url_summaries=project.url_summaries or None,
    )

script = asyncio.get_event_loop().run_until_complete(generate())

print(f"Title:    {script['video_title']}")
print(f"Style:    {script['video_style']}")
print(f"Duration: ~{script['target_duration_seconds']}s")
print(f"Segments: {len(script['segments'])}")

total_words = sum(len(s['narration_text'].split()) for s in script['segments'])
print(f"Words:    {total_words} (~{total_words * 60 // 150}s at 150 wpm)")

Title:    Play 200+ Games Free on One Portal
Style:    playful
Duration: ~45s
Segments: 5
Words:    57 (~22s at 150 wpm)


## 4. Inspect Segments

Review each segment's narration and FLUX scene description.

In [5]:
for seg in script['segments']:
    print(f"{'='*70}")
    print(f"Segment {seg['segment_id']}: {seg['segment_type'].upper()}")
    print(f"Visual: {seg['visual_style']}  |  Transition: {seg['transition']}")
    print(f"{'─'*70}")
    print(f"NARRATION ({len(seg['narration_text'].split())} words):")
    print(f"  \"{seg['narration_text']}\"")
    print()
    print(f"SCENE ({len(seg['scene_description'].split())} words):")
    print(f"  {seg['scene_description']}")
    print()

Segment 1: HOOK
Visual: vibrant  |  Transition: cut
──────────────────────────────────────────────────────────────────────
NARRATION (7 words):
  "200 games. One click. No downloads required."

SCENE (40 words):
  A giant glowing play button hovers above a vast digital sea of floating controller icons, each pulsing like fireflies. Soft cyan backlighting outlines the icons while ambient purple glow radiates from unseen servers below. Style: cinematic sci-fi. Mood: wonder, excitement.

Segment 2: INTRODUCTION
Visual: minimal  |  Transition: slide_left
──────────────────────────────────────────────────────────────────────
NARRATION (18 words):
  "Built by indie dev Alex Rivera using libGDX and threejs, this browser portal hosts over 200 free games."

SCENE (39 words):
  A hand-drawn notebook lies open on weathered oak, showing handwritten code schematics beside coffee stains. Morning light streams through a cracked window, illuminating dust motes and casting long warm golden shafts across

## 5. Raw JSON Output

The full script as JSON (this is what gets stored in `WaywoVideoDB.script_json`).

In [6]:
print(json.dumps(script, indent=2))

{
  "video_title": "Play 200+ Games Free on One Portal",
  "video_style": "playful",
  "target_duration_seconds": 45,
  "segments": [
    {
      "segment_id": 1,
      "segment_type": "hook",
      "narration_text": "200 games. One click. No downloads required.",
      "scene_description": "A giant glowing play button hovers above a vast digital sea of floating controller icons, each pulsing like fireflies. Soft cyan backlighting outlines the icons while ambient purple glow radiates from unseen servers below. Style: cinematic sci-fi. Mood: wonder, excitement.",
      "visual_style": "vibrant",
      "transition": "cut"
    },
    {
      "segment_id": 2,
      "segment_type": "introduction",
      "narration_text": "Built by indie dev Alex Rivera using libGDX and threejs, this browser portal hosts over 200 free games.",
      "scene_description": "A hand-drawn notebook lies open on weathered oak, showing handwritten code schematics beside coffee stains. Morning light streams through a

## 6. Regenerate for Variety

Run the same project again to see how the LLM varies its output. The creative temperature (0.8)
should produce a different script each time.

In [7]:
script2 = asyncio.get_event_loop().run_until_complete(generate())

print(f"First:  {script['video_title']} ({script['video_style']})")
print(f"Second: {script2['video_title']} ({script2['video_style']})")
print()
print("First hook: ", script['segments'][0]['narration_text'])
print("Second hook:", script2['segments'][0]['narration_text'])

First:  Play 200+ Games Free on One Portal (playful)
Second: Play 200+ Free Browser Games Instantly (playful)

First hook:  200 games. One click. No downloads required.
Second hook: Over 200 games, zero downloads — just click and play.


## Next Steps

- The generated script feeds into the video pipeline: TTS audio → STT timestamps → InvokeAI images → MoviePy assembly
- See `src/video_director.py` for the full prompt and validation logic
- Scene descriptions are optimized for FLUX 2 Klein (prose format, mandatory lighting, style/mood tags)